<a href="https://colab.research.google.com/github/SinghAnsh07/DataScience/blob/main/AI_Project_Fine_Tuned_VIT(Cancer_detection).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
# Extract to Colab's fast local storage
!unzip -q "/content/drive/MyDrive/BreaKHis_v1.zip" -d /content/

# Verify extraction
!ls /content/BreaKHis_v1/histology_slides/breast


benign	count_files.sh	malignant  README.txt


In [ ]:
!pip install timm -q
print("✓ Installation complete!")


✓ Installation complete!


In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.metrics import accuracy_score
# from tqdm import tqdm
import numpy as np
import random
import timm

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

print("✓ All libraries imported successfully!")


✓ All libraries imported successfully!


In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print("✓ GPU is ready for training!")
else:
    print("⚠️ Warning: No GPU detected. Training will be slow.")
    print("Enable GPU: Runtime → Change runtime type → T4 GPU")

# Configuration
data_root = '/content/BreaKHis_v1/histology_slides/breast'
epochs = 50
batch_size = 32
learning_rate = 1e-4

print(f"\nConfiguration:")
print(f"  Dataset path: {data_root}")
print(f"  Epochs: {epochs}")
print(f"  Batch size: {batch_size}")
print(f"  Learning rate: {learning_rate}")


Using device: cuda
GPU: Tesla T4
✓ GPU is ready for training!

Configuration:
  Dataset path: /content/BreaKHis_v1/histology_slides/breast
  Epochs: 50
  Batch size: 32
  Learning rate: 0.0001


In [ ]:
data_root = '/content/BreaKHis_v1/histology_slides/breast'


In [ ]:
# Verify dataset structure
print("Verifying dataset...")
print(f"\nChecking path: {data_root}")

benign_path = os.path.join(data_root, "benign")
malignant_path = os.path.join(data_root, "malignant")

if os.path.exists(benign_path) and os.path.exists(malignant_path):
    print("✓ Dataset folders found!")
    print(f"  - Benign folder: {benign_path}")
    print(f"  - Malignant folder: {malignant_path}")
else:
    print("❌ Dataset folders not found!")
    print("Check your data_root path")


Verifying dataset...

Checking path: /content/BreaKHis_v1/histology_slides/breast
✓ Dataset folders found!
  - Benign folder: /content/BreaKHis_v1/histology_slides/breast/benign
  - Malignant folder: /content/BreaKHis_v1/histology_slides/breast/malignant


In [ ]:
# Dataset Class
class BCDataset(Dataset):
    def __init__(self, files, labels, transform=None):
        self.files = files
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img = Image.open(self.files[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return img, label

print("✓ Dataset class created!")


✓ Dataset class created!


In [ ]:
# Data Augmentation for Training
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=30),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Data Transformation for Validation (no augmentation)
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("✓ Data transformations defined!")


✓ Data transformations defined!


In [ ]:
def load_data(magnification='100X'):
    """Load and prepare BreakHis dataset"""
    benign_path = os.path.join(data_root, "benign")
    malignant_path = os.path.join(data_root, "malignant")

    filenames, labels = [], []

    # Load images
    for path, label in [(benign_path, 0), (malignant_path, 1)]:
        for root, _, files in os.walk(path):
            if magnification in root:
                for file in files:
                    if file.lower().endswith('.png'):
                        filenames.append(os.path.join(root, file))
                        labels.append(label)

    print(f"\n{'='*60}")
    print(f"Dataset Loading Summary")
    print(f"{'='*60}")
    print(f"Magnification: {magnification}")
    print(f"Total images: {len(filenames)}")
    print(f"  - Benign: {labels.count(0)}")
    print(f"  - Malignant: {labels.count(1)}")

    # Shuffle and split (80% train, 20% validation)
    combined = list(zip(filenames, labels))
    random.shuffle(combined)
    filenames, labels = zip(*combined)

    split_idx = int(0.8 * len(filenames))
    train_files, val_files = filenames[:split_idx], filenames[split_idx:]
    train_labels, val_labels = labels[:split_idx], labels[split_idx:]

    print(f"  - Training set: {len(train_files)}")
    print(f"  - Validation set: {len(val_files)}")
    print(f"{'='*60}\n")

    # Create datasets
    train_dataset = BCDataset(train_files, train_labels, train_transform)
    val_dataset = BCDataset(val_files, val_labels, val_transform)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                             shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size,
                           shuffle=False, num_workers=2, pin_memory=True)

    return train_loader, val_loader

print("✓ Data loading function created!")


✓ Data loading function created!


In [ ]:
# ============================================================
# BREAKHIS TRAINING - ALL MAGNIFICATIONS
# Trains on 40X, 100X, 200X, and 400X
# Target: 99% accuracy on each magnification
# ============================================================

# Install library
!pip install timm -q

# Import libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tqdm import tqdm
import numpy as np
import random
import timm
from datetime import datetime

# Set seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}\n")

# Settings
data_root = '/content/BreaKHis_v1/histology_slides/breast'
epochs = 100
batch_size = 64
learning_rate = 3e-4

# All magnifications to train on
magnifications = ['40X', '100X', '200X', '400X']

# Store results
all_results = {}

# ============================================================
# DATASET CLASS
# ============================================================
class BCDataset(Dataset):
    def __init__(self, files, labels, transform=None):
        self.files = files
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img = Image.open(self.files[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, torch.tensor(self.labels[idx], dtype=torch.float32)

# ============================================================
# DATA AUGMENTATION
# ============================================================
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ============================================================
# LOAD DATA FUNCTION
# ============================================================
def load_data(magnification):
    benign_path = os.path.join(data_root, "benign")
    malignant_path = os.path.join(data_root, "malignant")

    filenames, labels = [], []
    for path, label in [(benign_path, 0), (malignant_path, 1)]:
        for root, _, files in os.walk(path):
            if magnification in root:
                for file in files:
                    if file.lower().endswith('.png'):
                        filenames.append(os.path.join(root, file))
                        labels.append(label)

    # Shuffle and split
    combined = list(zip(filenames, labels))
    random.shuffle(combined)
    filenames, labels = zip(*combined)

    split = int(0.8 * len(filenames))
    train_files, val_files = filenames[:split], filenames[split:]
    train_labels, val_labels = labels[:split], labels[split:]

    # Create data loaders
    train_dataset = BCDataset(train_files, train_labels, train_transform)
    val_dataset = BCDataset(val_files, val_labels, val_transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                             shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size,
                           shuffle=False, num_workers=2, pin_memory=True)

    return train_loader, val_loader, len(filenames), labels.count(0), labels.count(1)

# ============================================================
# TRAINING FUNCTION
# ============================================================
def train_magnification(magnification):
    print(f"\n{'='*70}")
    print(f"TRAINING: {magnification} MAGNIFICATION")
    print(f"{'='*70}\n")

    # Load data
    print(f"Loading {magnification} dataset...")
    train_loader, val_loader, total, benign, malignant = load_data(magnification)

    print(f"Total images: {total}")
    print(f"  Benign: {benign} | Malignant: {malignant}")
    print(f"  Train batches: {len(train_loader)} | Val batches: {len(val_loader)}\n")

    # Create model
    print(f"Creating ViT-Small model for {magnification}...")
    model = timm.create_model('vit_small_patch16_224', pretrained=True, num_classes=1)
    model.to(device)

    # Training setup
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-6)

    # Training loop
    print(f"Starting training for {epochs} epochs...\n")
    best_accuracy = 0.0

    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device).unsqueeze(1)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation phase
        model.eval()
        correct = 0
        total_samples = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = torch.sigmoid(model(images).squeeze())
                predicted = (outputs > 0.5).float()
                total_samples += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total_samples
        avg_loss = train_loss / len(train_loader)

        scheduler.step()

        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f} | Accuracy: {accuracy*100:.2f}%", end='')

        # Save best model
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            model_path = f'/content/drive/MyDrive/best_breakhis_{magnification}.pth'
            torch.save(model.state_dict(), model_path)
            print(f" ✓ BEST!")
        else:
            print()

    # Final evaluation
    print(f"\n{'─'*70}")
    print(f"FINAL EVALUATION - {magnification}")
    print(f"{'─'*70}")

    model.load_state_dict(torch.load(f'/content/drive/MyDrive/best_breakhis_{magnification}.pth'))
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            outputs = torch.sigmoid(model(images).squeeze())
            predicted = (outputs > 0.5).float()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())

    # Classification report
    print(classification_report(all_labels, all_preds,
                               target_names=['Benign', 'Malignant'],
                               digits=4))

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    print(f"\nConfusion Matrix:")
    print(f"                Predicted")
    print(f"              Benign  Malignant")
    print(f"Actual Benign   {cm[0][0]:4d}      {cm[0][1]:4d}")
    print(f"    Malignant   {cm[1][0]:4d}      {cm[1][1]:4d}")

    print(f"\n✓ {magnification} Training Complete!")
    print(f"Best Accuracy: {best_accuracy*100:.2f}%")
    print(f"Model saved: /content/drive/MyDrive/best_breakhis_{magnification}.pth")

    return best_accuracy, cm

# ============================================================
# TRAIN ALL MAGNIFICATIONS
# ============================================================
print(f"\n{'#'*70}")
print(f"# BREAKHIS BREAST CANCER CLASSIFICATION - ALL MAGNIFICATIONS")
print(f"# Training on: 40X, 100X, 200X, 400X")
print(f"# Target: 99% accuracy on each magnification")
print(f"{'#'*70}")

start_time = datetime.now()

for mag in magnifications:
    accuracy, cm = train_magnification(mag)
    all_results[mag] = {
        'accuracy': accuracy,
        'confusion_matrix': cm
    }

end_time = datetime.now()
training_duration = end_time - start_time

# ============================================================
# FINAL SUMMARY REPORT
# ============================================================
print(f"\n\n{'='*70}")
print(f"COMPLETE TRAINING SUMMARY - ALL MAGNIFICATIONS")
print(f"{'='*70}\n")

print(f"Training Duration: {training_duration}\n")

print(f"{'Magnification':<15} {'Accuracy':<15} {'Status'}")
print(f"{'-'*70}")

for mag in magnifications:
    accuracy = all_results[mag]['accuracy'] * 100
    status = "✓ EXCELLENT" if accuracy >= 99 else "✓ GOOD" if accuracy >= 97 else "⚠ NEEDS IMPROVEMENT"
    print(f"{mag:<15} {accuracy:>6.2f}%{'':<8} {status}")

print(f"\n{'-'*70}")

# Average accuracy
avg_accuracy = np.mean([all_results[mag]['accuracy'] * 100 for mag in magnifications])
print(f"{'Average':<15} {avg_accuracy:>6.2f}%")

print(f"\n{'='*70}")
print(f"MODELS SAVED TO GOOGLE DRIVE:")
print(f"{'='*70}")
for mag in magnifications:
    print(f"  • best_breakhis_{mag}.pth")

print(f"\n{'='*70}")
print(f"✓ ALL MAGNIFICATIONS TRAINING COMPLETE!")
print(f"Target (Gella 2024): 99.99% per magnification")
print(f"Your Results: See table above")
print(f"{'='*70}\n")


Device: cuda


######################################################################
# BREAKHIS BREAST CANCER CLASSIFICATION - ALL MAGNIFICATIONS
# Training on: 40X, 100X, 200X, 400X
# Target: 99% accuracy on each magnification
######################################################################

TRAINING: 40X MAGNIFICATION

Loading 40X dataset...
Total images: 1995
  Benign: 625 | Malignant: 1370
  Train batches: 25 | Val batches: 7

Creating ViT-Small model for 40X...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/88.2M [00:00<?, ?B/s]

Starting training for 100 epochs...



40X Epoch 1/100: 100%|██████████| 25/25 [00:36<00:00,  1.46s/it]


...

Epoch 98/100 - Loss: 0.0001 | Accuracy: 98.90%


400X Epoch 99/100: 100%|██████████| 23/23 [00:26<00:00,  1.13s/it]


Epoch 99/100 - Loss: 0.0001 | Accuracy: 98.90%


400X Epoch 100/100: 100%|██████████| 23/23 [00:25<00:00,  1.11s/it]


Epoch 100/100 - Loss: 0.0001 | Accuracy: 98.90%

──────────────────────────────────────────────────────────────────────
FINAL EVALUATION - 400X
──────────────────────────────────────────────────────────────────────
              precision    recall  f1-score   support

      Benign     0.9837    0.9918    0.9878       122
   Malignant     0.9959    0.9917    0.9938       242

    accuracy                         0.9918       364
   macro avg     0.9898    0.9918    0.9908       364
weighted avg     0.9918    0.9918    0.9918       364


Confusion Matrix:
                Predicted
              Benign  Malignant
Actual Benign    121         1
    Malignant      2       240

✓ 400X Training Complete!
Best Accuracy: 99.18%
Model saved: /content/drive/MyDrive/best_breakhis_400X.pth


COMPLETE TRAINING SUMMARY - ALL MAGNIFICATIONS

Training Duration: 3:46:55.305447

Magnification   Accuracy        Status
----------------------------------------------------------------------
40X             